In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !apt-get install sumo sumo-tools sumo-doc

In [ ]:
# !pip install sumo_rl
# !pip install torch torchrl 'ray[rllib]' 'ray[tune]'
# !pip install pettingzoo
# !pip install tensorboard
# !pip install wandb

In [ ]:
# %env SUMO_HOME=/usr/share/sumo

In [ ]:
# DIR_PATH="/content/drive/MyDrive/sumo_pettingzoo_rllib"

In [ ]:
import sumo_rl
import ray
import wandb
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray import tune
from ray.tune.registry import register_env
from ray.rllib.env import PettingZooEnv
from torchrl.envs.libs.pettingzoo import PettingZooWrapper

In [ ]:
wandb.login()

In [ ]:
wandb.tensorboard.patch(root_logdir=DIR_PATH+"/ray_results")

In [ ]:
wandb.init(
    project="sumo_petting_zoo_rllib",
)

In [ ]:
ray.shutdown()
ray.init()

In [ ]:
env_pz = ParallelPettingZooEnv(sumo_rl.parallel_env(
            net_file=DIR_PATH+'/net/grid4x4/grid4x4.net.xml',
            route_file=DIR_PATH+'/net/grid4x4/grid4x4_1.rou.xml',
            use_gui=False,
            num_seconds=2000))

def env_creator(config):
    return env_pz

register_env("SumoEnv", lambda config: env_creator(config))

env_pz.close()


In [ ]:
agents = [a for a in env_pz.get_agent_ids()]

In [ ]:
def custom_policy_fn(agent_id):
    return (None, env_pz.observation_space[agent_id], env_pz.action_space[agent_id], {})

def policy_mapping(agent_id, episode, worker, **kwards):
    return agent_id


In [ ]:
config = {
    "env": "SumoEnv",
    "multiagent": {
        "policies": {
            agent_id: custom_policy_fn(agent_id)
            for agent_id in agents
        },
        "policy_mapping_fn": policy_mapping,
    },
    "framework": "torch",
    "num_cpus_per_worker": 1,
    "resources_per_trial": 2,
    "num_workers": 1,
    "horizon": 1500,
    "soft_horizon": False
}

In [ ]:
tune.run(
    "PPO",
    config=config,
    stop={"timesteps_total": 100000},
    checkpoint_at_end=True,
    local_dir=DIR_PATH+"/ray_results",
)

In [ ]:
wandb.finish()